In [188]:
import numpy
import pandas as pd
import csv
import matplotlib.pyplot as plt
import seaborn as sns

In [189]:
imotions_emo_emotions = ['Joy', 'Anger', 'Surprise', 'Fear', 'Contempt', 'Disgust', 'Sadness', 'Confusion', 'Frustration']
imotions_emo_emotions.sort()
imotions_emo_valence = ['Neutral', 'Positive', 'Negative']

imotions_aff4_emotions = ['Anger', 'Contempt', 'Disgust', 'Engagement', 'Fear', 'Joy', 'Sadness', 'Surprise']
imotions_aff4_emotions.sort()
imotions_aff4_valence = ['Valence']

imotions_aff5_emotions = ['Anger', 'Contempt', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise', 'Engagement', 'Sentimentality', 'Confusion', 'Neutral']
#imotions_aff5_emotions = ['Anger', 'Contempt', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise', 'Engagement', 'Sentimentality', 'Confusion']
imotions_aff5_emotions.sort()
imotions_aff5_valence = ['Valence']

pyfeat_emotions = ['anger', 'disgust', 'fear', 'happiness', 'sadness', 'surprise', 'neutral']
pyfeat_emotions.sort()
#pyfeat_emotions = ['anger', 'disgust', 'fear', 'happiness', 'sadness', 'surprise']

libreface_emotions = ['facial_expression']

In [229]:
def preprocess_imotions(imotions_file, algorithm):
    data_starts = imotions_file[imotions_file['#INFO'] == '#DATA'].index[0]
    imotions_file = imotions_file.loc[data_starts+1:]
    new_header = imotions_file.iloc[0] #grab the first row for the header
    imotions_file = imotions_file[1:] #take the data less the header row
    imotions_file.columns = new_header #set the header row as the df header
    imotions_file = imotions_file.iloc[2:]
    imotions_file = imotions_file.iloc[:-2] # there are also two random rows at the end we want to drop

    pspi_au_imotions_aff = ['Brow Furrow', 'Cheek Raise', 'Lid Tighten', 'Nose Wrinkle', 'Upper Lip Raise', 'Eye Closure']
    
    my_cols_imotions = []
    for c in imotions_file.columns:
        if 'au' in c or "AU" in c:
            my_cols_imotions.append(c) 
    if algorithm == 'emo':
        my_cols_imotions.extend(imotions_emo_emotions)
        my_cols_imotions.extend(imotions_emo_valence)
    elif algorithm == 'aff4':
        my_cols_imotions.extend(imotions_aff4_emotions) 
        my_cols_imotions.extend(imotions_aff4_valence)
        my_cols_imotions.extend(pspi_au_imotions_aff)       
    elif algorithm == 'aff5':
        my_cols_imotions.extend(imotions_aff5_emotions) 
        my_cols_imotions.extend(imotions_aff5_valence)
        my_cols_imotions.extend(pspi_au_imotions_aff)
    
    imotions_file = imotions_file[my_cols_imotions]

    if algorithm == 'emo':    
        dict_imotions = {}
        for c in imotions_file.columns:
             dict_imotions[c] = 'imotions_' + c.lower()        
        dict_imotions['AU1'] = 'imotions_au01'
        dict_imotions['AU2'] = 'imotions_au02'
        dict_imotions['AU4'] = 'imotions_au04'
        dict_imotions['AU5'] = 'imotions_au05'
        dict_imotions['AU6'] = 'imotions_au06'
        dict_imotions['AU7'] = 'imotions_au07'
        dict_imotions['AU9'] = 'imotions_au09'
        imotions_file = imotions_file.rename(columns=dict_imotions)
    elif algorithm == 'aff4' or algorithm == 'aff5':
        dict_imotions = {'Brow Furrow':'imotions_au04', 
                         'Cheek Raise':'imotions_au06', 
                         'Lid Tighten':'imotions_au07', 
                         'Nose Wrinkle':'imotions_au09', 
                         'Upper Lip Raise':'imotions_au10', 
                         'Eye Closure':'imotions_au43'}
        imotions_file = imotions_file.rename(columns=dict_imotions)        
    return(imotions_file)

def preprocess_pyfeat(pyfeat_file):
    my_cols_pyfeat = []
    for c in pyfeat_file.columns:
        if 'au' in c or "AU" in c:
            my_cols_pyfeat.append(c)
    my_cols_pyfeat.extend(pyfeat_emotions)
    mydat_pyfeat = pyfeat_file[my_cols_pyfeat]
    dict_pf = {}
    for c in mydat_pyfeat.columns:
        dict_pf[c] = 'pyfeat_' + c.lower()

    mydat_pyfeat = mydat_pyfeat.rename(columns=dict_pf)
    return(mydat_pyfeat)

def preprocess_libreface(lf_file):
    my_cols_libreface = []
    for c in lf_file.columns:
        if 'au' in c or "AU" in c:
            my_cols_libreface.append(c)
    my_cols_libreface.extend(['facial_expression'])
    mydat_libreface = lf_file[my_cols_libreface]
    dict_lf = {}
    for c in mydat_libreface.columns:
        if 'au' in c:
            dict_lf[c] = 'libreface_au' + c.lower()[3:]
    dict_lf['au_1'] = 'libreface_au01'
    dict_lf['au_2'] = 'libreface_au02'
    dict_lf['au_4'] = 'libreface_au04'
    dict_lf['au_6'] = 'libreface_au06'
    dict_lf['au_7'] = 'libreface_au07'

    dict_lf['au_1_intensity'] = 'libreface_au01_intensity'
    dict_lf['au_2_intensity'] = 'libreface_au02_intensity'
    dict_lf['au_4_intensity'] = 'libreface_au04_intensity'
    dict_lf['au_5_intensity'] = 'libreface_au05_intensity'
    dict_lf['au_6_intensity'] = 'libreface_au06_intensity'
    dict_lf['au_9_intensity'] = 'libreface_au09_intensity'
    
    mydat_libreface = mydat_libreface.rename(columns=dict_lf)
    return(mydat_libreface)

In [232]:
def preprocess_imotions2(imotions_file, algorithm):
    data_starts = imotions_file[imotions_file['#INFO'] == '#DATA'].index[0]
    imotions_file = imotions_file.loc[data_starts+1:]
    new_header = imotions_file.iloc[0] #grab the first row for the header
    imotions_file = imotions_file[1:] #take the data less the header row
    imotions_file.columns = new_header #set the header row as the df header
    imotions_file = imotions_file.iloc[2:]
    imotions_file = imotions_file.iloc[:-2] # there are also two random rows at the end we want to drop

    my_cols_imotions = []
    for c in imotions_file.columns:
        if 'au' in c or "AU" in c:
            my_cols_imotions.append(c) 
    if algorithm == 'emo':
        my_cols_imotions.extend(imotions_emo_emotions)
        my_cols_imotions.extend(imotions_emo_valence)
    elif algorithm == 'aff4':
        my_cols_imotions.extend(imotions_aff4_emotions) 
        my_cols_imotions.extend(imotions_aff4_valence)
    elif algorithm == 'aff5':
        my_cols_imotions.extend(imotions_aff5_emotions) 
        my_cols_imotions.extend(imotions_aff5_valence)
    imotions_file = imotions_file[my_cols_imotions]

    return(imotions_file)


def preprocess_pyfeat2(pyfeat_file):
    my_cols_pyfeat = []
    for c in pyfeat_file.columns:
        if 'au' in c or "AU" in c:
            my_cols_pyfeat.append(c)
    my_cols_pyfeat.extend(['anger', 'disgust', 'fear', 'happiness', 'sadness', 'surprise', 'neutral'])
    mydat_pyfeat = pyfeat_file[my_cols_pyfeat]
    return(mydat_pyfeat)

def preprocess_libreface2(lf_file):
    my_cols_libreface = []
    for c in lf_file.columns:
        if 'au' in c or "AU" in c:
            my_cols_libreface.append(c)
    my_cols_libreface.extend(['facial_expression'])
    mydat_libreface = lf_file[my_cols_libreface]
    return(mydat_libreface)

def run_emotion_comparison(videoname, do_aff4 = True, do_aff5 = True, do_emo = True, 
                           do_pyfeat_svm = True, do_pyfeat_xgb = True, do_libreface = True):
    # load files
    path_to_folder = '/Users/zeleninam2/Documents/1_projects/1_FACE_PAIN/FER_installs/my_imgs/outputs/' + videoname +'/'
    if do_aff4:
        dat_imotions_aff4 = pd.read_csv(path_to_folder + videoname + '_imotions_aff4.csv')
        mydat_imotions_aff4 = preprocess_imotions2(dat_imotions_aff4, algorithm = 'aff4')
    if do_aff5:
        dat_imotions_aff5 = pd.read_csv(path_to_folder + videoname + '_imotions_aff5-1.csv')  
        mydat_imotions_aff5 = preprocess_imotions2(dat_imotions_aff5, algorithm = 'aff5')
    if do_emo:
        dat_imotions_emotient = pd.read_csv(path_to_folder + videoname + '_imotions_emotient.csv')
        mydat_imotions_emotient = preprocess_imotions2(dat_imotions_emotient, algorithm = 'emo')
    if do_pyfeat_xgb:
        dat_pyfeat_xgb = pd.read_csv(path_to_folder + videoname + '_pyfeat_xgb.csv')
        mydat_pyfeat_xgb = preprocess_pyfeat2(dat_pyfeat_xgb)
    if do_pyfeat_svm:
        dat_pyfeat_svm = pd.read_csv(path_to_folder + videoname + '_pyfeat_svm.csv')
        mydat_pyfeat_svm = preprocess_pyfeat2(dat_pyfeat_svm)
    if do_libreface:
        dat_libreface = pd.read_csv(path_to_folder + videoname + '_libreface.csv')
        mydat_libreface = preprocess_libreface2(dat_libreface)

    # run comparison
    # just print the stats for now

    if do_aff4:
        print('imotions - aff4')
        em_l = [item.lower() for item in imotions_aff4_emotions]
        print('EMOTIONS (alphabetical):')
        print(em_l)
        print('\nVALENCE: valence')
        print('\nSTATS:')   
        mydat_imotions_aff4['max_emotion'] = mydat_imotions_aff4[imotions_aff4_emotions].idxmax(axis=1)
        print(mydat_imotions_aff4.max_emotion.value_counts())
        print(mydat_imotions_aff4.Valence.value_counts())
        print('out of: ' + str(mydat_imotions_aff4.shape[0]))

    if do_aff5:
        print('\nimotions - aff5.1\n')
        em_l = [item.lower() for item in imotions_aff5_emotions]
        print('EMOTIONS (alphabetical):')
        print(em_l)
        print('\nVALENCE: valence')
        print('\nSTATS:')         
        mydat_imotions_aff5['max_emotion'] = mydat_imotions_aff5[imotions_aff5_emotions].idxmax(axis=1)
        print(mydat_imotions_aff5.max_emotion.value_counts())
        print(mydat_imotions_aff5.Valence.value_counts())
        print('out of: ' + str(mydat_imotions_aff5.shape[0]))
        
    if do_emo:
        print('\nimotions - emotient\n')
        em_l = [item.lower() for item in imotions_emo_emotions]
        print('EMOTIONS (alphabetical):')
        print(em_l)        
        print('\nVALENCE: negative, neutral, positive')
        print('\nSTATS:') 
        mydat_imotions_emotient['max_emotion'] = mydat_imotions_emotient[imotions_emo_emotions].idxmax(axis=1)
        mydat_imotions_emotient['max_valence'] = mydat_imotions_emotient[imotions_emo_valence].idxmax(axis=1)
        print(mydat_imotions_emotient.max_emotion.value_counts())
        print(mydat_imotions_emotient.max_valence.value_counts())
        print('out of: ' + str(mydat_imotions_emotient.shape[0]))

    if do_pyfeat_xgb:
        print('\npyfeat - xgb\n')
        em_l = [item.lower() for item in pyfeat_emotions]
        print('EMOTIONS (alphabetical):')
        print(em_l)
        print('\nVALENCE: NO')
        print('\nSTATS:')  
        mydat_pyfeat_xgb['max_emotion'] = mydat_pyfeat_xgb[pyfeat_emotions].idxmax(axis=1)
        print(mydat_pyfeat_xgb.max_emotion.value_counts())
        print('out of: ' + str(mydat_pyfeat_xgb.shape[0]))
        
    if do_pyfeat_svm:
        print('\npyfeat - svm\n')
        em_l = [item.lower() for item in pyfeat_emotions]
        print('EMOTIONS (alphabetical):')
        print(em_l)
        print('\nVALENCE: NO')
        print('\nSTATS:') 
        mydat_pyfeat_svm['max_emotion'] = mydat_pyfeat_svm[pyfeat_emotions].idxmax(axis=1)
        print(mydat_pyfeat_svm.max_emotion.value_counts())
        print('out of: ' + str(mydat_pyfeat_svm.shape[0]))
        
    if do_libreface:
        print('\nlibreface\n')
        print('EMOTIONS (alphabetical):\n')
        print('Just a column "facial_expression", they dont report which ones')
        print('\nVALENCE: NO')
        print('\nSTATS:') 
        print(mydat_libreface.facial_expression.value_counts())
        print('out of: ' + str(mydat_libreface.shape[0]))
        
    return(0)



In [233]:
my_videoname = '9697_heat1_trial04'
run_emotion_comparison(videoname = my_videoname, do_aff4 = False, do_aff5 = False)


imotions - emotient

EMOTIONS (alphabetical):
['anger', 'confusion', 'contempt', 'disgust', 'fear', 'frustration', 'joy', 'sadness', 'surprise']

VALENCE: negative, neutral, positive

STATS:
max_emotion
Frustration    223
Disgust        182
Confusion      121
Contempt        63
Anger            8
Sadness          4
Name: count, dtype: int64
max_valence
Negative    601
Name: count, dtype: int64
out of: 601

pyfeat - xgb

EMOTIONS (alphabetical):
['anger', 'disgust', 'fear', 'happiness', 'neutral', 'sadness', 'surprise']

VALENCE: NO

STATS:


/var/folders/d8/s1szglf95wl4_7_f5jkmn561n3477x/T/ipykernel_70554/3790804062.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mydat_pyfeat_xgb['max_emotion'] = mydat_pyfeat_xgb[pyfeat_emotions].idxmax(axis=1)


max_emotion
sadness    601
Name: count, dtype: int64
out of: 601

pyfeat - svm

EMOTIONS (alphabetical):
['anger', 'disgust', 'fear', 'happiness', 'neutral', 'sadness', 'surprise']

VALENCE: NO

STATS:


/var/folders/d8/s1szglf95wl4_7_f5jkmn561n3477x/T/ipykernel_70554/3790804062.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mydat_pyfeat_svm['max_emotion'] = mydat_pyfeat_svm[pyfeat_emotions].idxmax(axis=1)


max_emotion
sadness    601
Name: count, dtype: int64
out of: 601

libreface

EMOTIONS (alphabetical):

Just a column "facial_expression", they dont report which ones

VALENCE: NO

STATS:
facial_expression
Anger      235
Neutral      7
Name: count, dtype: int64
out of: 242


0

In [234]:
my_videoname = '10479_heat1_trial09'
run_emotion_comparison(videoname = my_videoname, do_libreface = False)

imotions - aff4
EMOTIONS (alphabetical):
['anger', 'contempt', 'disgust', 'engagement', 'fear', 'joy', 'sadness', 'surprise']

VALENCE: valence

STATS:
max_emotion
Disgust    600
Name: count, dtype: int64
Valence
0    600
Name: count, dtype: int64
out of: 600

imotions - aff5.1

EMOTIONS (alphabetical):
['anger', 'confusion', 'contempt', 'disgust', 'engagement', 'fear', 'joy', 'neutral', 'sadness', 'sentimentality', 'surprise']

VALENCE: valence

STATS:
max_emotion
Neutral    600
Name: count, dtype: int64
Valence
0    600
Name: count, dtype: int64
out of: 600

imotions - emotient

EMOTIONS (alphabetical):
['anger', 'confusion', 'contempt', 'disgust', 'fear', 'frustration', 'joy', 'sadness', 'surprise']

VALENCE: negative, neutral, positive

STATS:
max_emotion
Sadness        318
Contempt       254
Frustration     23
Surprise         5
Name: count, dtype: int64
max_valence
Neutral     389
Negative    208
Positive      3
Name: count, dtype: int64
out of: 600

pyfeat - xgb

EMOTIONS (alpha

/var/folders/d8/s1szglf95wl4_7_f5jkmn561n3477x/T/ipykernel_70554/3790804062.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mydat_pyfeat_xgb['max_emotion'] = mydat_pyfeat_xgb[pyfeat_emotions].idxmax(axis=1)


max_emotion
neutral    600
Name: count, dtype: int64
out of: 600

pyfeat - svm

EMOTIONS (alphabetical):
['anger', 'disgust', 'fear', 'happiness', 'neutral', 'sadness', 'surprise']

VALENCE: NO

STATS:


/var/folders/d8/s1szglf95wl4_7_f5jkmn561n3477x/T/ipykernel_70554/3790804062.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mydat_pyfeat_svm['max_emotion'] = mydat_pyfeat_svm[pyfeat_emotions].idxmax(axis=1)


max_emotion
neutral    600
Name: count, dtype: int64
out of: 600


0

# compare AUs

In [235]:
videoname_9697 = '9697_heat1_trial04'
videoname_10479 = '10479_heat1_trial09'

path_to_folder_9697 = '/Users/zeleninam2/Documents/1_projects/1_FACE_PAIN/FER_installs/my_imgs/outputs/' + videoname_9697 +'/'
path_to_folder_10479 = '/Users/zeleninam2/Documents/1_projects/1_FACE_PAIN/FER_installs/my_imgs/outputs/' + videoname_10479 +'/'

# 9697

dat_9697_imotions_emotient = pd.read_csv(path_to_folder_9697 + videoname_9697 + '_imotions_emotient.csv')
mydat_9697_imotions_emotient = preprocess_imotions(dat_9697_imotions_emotient, algorithm = 'emo')

dat_9697_pyfeat_xgb = pd.read_csv(path_to_folder_9697 + videoname_9697 + '_pyfeat_xgb.csv')
mydat_9697_pyfeat_xgb = preprocess_pyfeat(dat_9697_pyfeat_xgb)

dat_9697_pyfeat_svm = pd.read_csv(path_to_folder_9697 + videoname_9697 + '_pyfeat_svm.csv')
mydat_9697_pyfeat_svm = preprocess_pyfeat(dat_9697_pyfeat_svm)

dat_9697_libreface = pd.read_csv(path_to_folder_9697 + videoname_9697 + '_libreface.csv')
mydat_9697_libreface = preprocess_libreface(dat_9697_libreface)


# 10479

dat_10479_imotions_aff4 = pd.read_csv(path_to_folder_10479 + videoname_10479 + '_imotions_aff4.csv')
mydat_10479_imotions_aff4 = preprocess_imotions(dat_10479_imotions_aff4, algorithm = 'aff4')

dat_10479_imotions_aff5 = pd.read_csv(path_to_folder_10479 + videoname_10479 + '_imotions_aff5-1.csv')
mydat_10479_imotions_aff5 = preprocess_imotions(dat_10479_imotions_aff5, algorithm = 'aff5')

dat_10479_imotions_emotient = pd.read_csv(path_to_folder_10479 + videoname_10479 + '_imotions_emotient.csv')
mydat_10479_imotions_emotient = preprocess_imotions(dat_10479_imotions_emotient, algorithm = 'emo')

dat_10479_pyfeat_xgb = pd.read_csv(path_to_folder_10479 + videoname_10479 + '_pyfeat_xgb.csv')
mydat_10479_pyfeat_xgb = preprocess_pyfeat(dat_10479_pyfeat_xgb)

dat_10479_pyfeat_svm = pd.read_csv(path_to_folder_10479 + videoname_10479 + '_pyfeat_svm.csv')
mydat_10479_pyfeat_svm = preprocess_pyfeat(dat_10479_pyfeat_svm)


## only looking at PSPI AUs

In [237]:
# pspi = au4, au6, au7, au9, au10, au43

#### 9697

In [238]:
# how do different datafiles look like?

In [239]:
mydat_9697_imotions_emotient = mydat_9697_imotions_emotient[['imotions_au04', 'imotions_au06', 'imotions_au07', 'imotions_au09', 'imotions_au10', 'imotions_au43']]
mydat_9697_imotions_emotient.head(1)

23,imotions_au04,imotions_au06,imotions_au07,imotions_au09,imotions_au10,imotions_au43
26,-0.23285722732543945,0.2272878885269165,0.24637952446937561,-0.87159746885299683,0.16947807371616364,-0.33074581623077393


In [244]:
mydat_9697_imotions_emotient.max()

23
imotions_au04    0.52918553352355957
imotions_au06     1.0798442363739014
imotions_au07     0.9679836630821228
imotions_au09    0.30778861045837402
imotions_au10    0.57795417308807373
imotions_au43    -1.5189880132675171
dtype: object

In [240]:
mydat_9697_pyfeat_xgb = mydat_9697_pyfeat_xgb[['pyfeat_au04', 'pyfeat_au06', 'pyfeat_au07', 'pyfeat_au09', 'pyfeat_au10', 'pyfeat_au43']]
mydat_9697_pyfeat_xgb.head(1)

,pyfeat_au04,pyfeat_au06,pyfeat_au07,pyfeat_au09,pyfeat_au10,pyfeat_au43
0,0.673706,0.221653,0.0,0.781691,0.12046,0.503665


In [241]:
mydat_9697_pyfeat_svm = mydat_9697_pyfeat_svm[['pyfeat_au04', 'pyfeat_au06', 'pyfeat_au07', 'pyfeat_au09', 'pyfeat_au10', 'pyfeat_au43']]
mydat_9697_pyfeat_svm.head(1)

,pyfeat_au04,pyfeat_au06,pyfeat_au07,pyfeat_au09,pyfeat_au10,pyfeat_au43
0,1.0,0.0,0.0,1.0,0.0,1.0


In [243]:
mydat_9697_libreface = mydat_9697_libreface[['libreface_au01','libreface_au02','libreface_au04','libreface_au06', 'libreface_au07',
                                            'libreface_au01_intensity', 'libreface_au02_intensity', 'libreface_au04_intensity', 'libreface_au05_intensity', 'libreface_au06_intensity','libreface_au09_intensity']]
mydat_9697_libreface.head(1)

,libreface_au01,libreface_au02,libreface_au04,libreface_au06,libreface_au07,libreface_au01_intensity,libreface_au02_intensity,libreface_au04_intensity,libreface_au05_intensity,libreface_au06_intensity,libreface_au09_intensity
0,0,0,1,1,1,0.078946,0.061479,0.716611,0.047985,0.163758,0.055708


In [ ]:
# super simple comparison - pyfeat vs libreface - present or not present, binary